In [622]:
import os
from os import environ

import requests
import io

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt

import calendar
import time

In [623]:
#route = environ['route']
destination='ACE'
path='/home/ubuntu/Data/'
downweight_granularity=['FltNum','Direction','dtime','dday','month']

# Data Import

In [624]:
#Data=pd.read_csv(path+'SQLsnippets_tours/SQLsnippet_'+destination+'_allprod.csv',sep=',')
Data=pd.read_csv('SQLsnippet_'+destination+'_allprod.csv',sep=',')
Data['calendar_date']=pd.to_datetime(Data['calendar_date'])
Data['pax_total']=Data['pax_total'].fillna(0)
Data['group_pax']=Data['group_pax'].fillna(0)
#Capa=pd.read_csv(path+'SQLsnippets_tours/SQLsnippet_'+destination+'_capa.csv',sep=',')
#Capa['calendar_date']=pd.to_datetime(Capa['calendar_date'])

Data=Data[[x for x in Data.columns if x!='dprio']+['dprio']]

AllFlights=Data[['calendar_date','dep_time','flight_number','airport_arrival','airport_depart']].drop_duplicates()
AllFlights.index=range(AllFlights.shape[0])
AllTime=[x for x in range(0,367)]

AllTime=Series(AllTime*AllFlights.shape[0])

AllFlights=AllFlights.loc[np.repeat(AllFlights.index,len(AllTime)/AllFlights.shape[0]),:]
AllFlights.index=range(AllFlights.shape[0])

AllFlights=pd.concat([AllFlights,Series([0]*len(AllTime)),Series([0]*len(AllTime)),AllTime],ignore_index=True,axis=1)

AllFlights.columns=Data.columns

Data=pd.concat([Data,AllFlights])

Data=Data.drop_duplicates(subset=['calendar_date','flight_number','dep_time','dprio'])

Data=Data.groupby(['calendar_date','airport_depart','airport_arrival','dprio'])[['pax_total','group_pax']].sum().reset_index()

Data['diff']=(Data['calendar_date']-pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))).apply(lambda x: x.days)

Data=Data.loc[Data['dprio']>Data['diff'],:]

Data=Data[[x for x in Data.columns if x!='diff']]

Data=Data[['calendar_date','airport_depart','airport_arrival','dprio','pax_total','group_pax']]\
.sort_values(by=['calendar_date','dprio'],ascending=[True,False])

Data=Data.set_index(['calendar_date','airport_depart','airport_arrival','dprio'])

Data=Data.groupby(level=[0,1,2]).diff().reset_index()
Data=Data.dropna(how='any')

Data['weekday']=Data['calendar_date'].apply(lambda x: list(calendar.day_name)[x.weekday()])
Data['route']='LUX-'+destination
Data['direction']='O'
Data.loc[Data['airport_arrival']=='LUX','direction']='I'
Data=Data[['calendar_date','weekday','route','direction','dprio','pax_total','group_pax']]

In [625]:
#Holiday_LUX=pd.read_csv('/home/ubuntu/Data/holiday/LUX-LUX_special_days.csv',encoding="ISO-8859-1")
Holiday_LUX=pd.read_csv('LUX-LUX_special_days.csv',encoding="ISO-8859-1")
Holiday_LUX['date']=pd.to_datetime(Holiday_LUX['date'])
Holiday_LUX['nom_jour_ferie']=Holiday_LUX['nom_jour_ferie'].apply(lambda x: 'Luxembourg' if 'Luxembourg' in str(x) else x)
Holiday_LUX['nom_jour_ferie']=Holiday_LUX['nom_jour_ferie'].apply(lambda x: 'Travail' if 'travail' in str(x) else x)
#Holiday_SCN=pd.read_csv('/home/ubuntu/Data/holiday/LUX-SCN_special_days.csv',encoding="ISO-8859-1")
Holiday_SCN=pd.read_csv('LUX-SCN_special_days.csv',encoding="ISO-8859-1")
Holiday_SCN['date']=pd.to_datetime(Holiday_SCN['date'])
#Holiday_SXB=pd.read_csv('/home/ubuntu/Data/holiday/LUX-SXB_special_days.csv',encoding="ISO-8859-1")
Holiday_SXB=pd.read_csv('LUX-SXB_special_days.csv',encoding="ISO-8859-1")
Holiday_SXB['date']=pd.to_datetime(Holiday_SXB['date'])
#Holiday_BRU=pd.read_csv('/home/ubuntu/Data/holiday/LUX-BRU_special_days.csv',encoding="ISO-8859-1")
Holiday_BRU=pd.read_csv('LUX-BRU_special_days.csv',encoding="ISO-8859-1")
Holiday_BRU['date']=pd.to_datetime(Holiday_BRU['date'])

#Holiday_LUX['nom_jour_ferie']=Holiday_LUX['nom_jour_ferie'].fillna('None')
#Holiday_LUX['nom_vacances']=Holiday_LUX['nom_vacances'].fillna('None')

In [626]:
Holiday=DataFrame(Holiday_LUX['date'])
Holiday['weekday']=Holiday['date'].apply(lambda x: list(calendar.day_name)[x.weekday()])
Holiday['yday']=Holiday['date'].apply(lambda x: x.timetuple().tm_yday)

In [627]:
#for country in ['LUX','FR','BE','DE']:
for country in ['LUX']:

    if country=='LUX':
        holiday=Holiday_LUX
    elif country=='FR':
        holiday=Holiday_SCN
    elif country=='BE':
        holiday=Holiday_BRU
    elif country=='DE':
        holiday=Holiday_SXB

        
    holiday['seconds']=holiday['date'].apply(lambda x: time.mktime(x.timetuple())/86400)
    holiday['yday']=holiday['date'].apply(lambda x: x.timetuple().tm_yday)

    holiday['whichholiday_left']='None'
    holiday['holiday_from_left']=0
    holiday['holiday_from_right']=0
    holiday['holiday_to_left']=0
    holiday['holiday_to_right']=0
    holiday['whichholiday_right']='None'
    holiday['whichjf_left']='None'
    holiday['jf_left']=0
    holiday['jf_right']=0
    holiday['whichjf_right']='None'


    def diff_smooth_1(x):
        dist_right=[y for y in x if y<0]
        if(len(dist_right)>0):
            min_right=max(dist_right)
            return abs(min_right)
        else:
            return np.nan

    def diff_smooth_2(x):
        dist_left=[y for y in x if y>0]
        if(len(dist_left)>0):
            min_left=min(dist_left)
            return abs(min_left)
        else:
            return np.nan

    smooth_list=[['Dimanche','Lundi'],['Samedi','Lundi'],['Samedi','Vendredi'],['Dimanche','Vendredi']]

    for l in smooth_list:

        smooth_from=l[0]
        smooth_til=l[1]
        holiday_distsmooth=np.round(DataFrame(np.subtract.outer(holiday.loc[(~holiday['vacances'])&(holiday['Weekday']==smooth_from),'seconds'],
        holiday.loc[(holiday['vacances'])&(holiday['Weekday']==smooth_til),'seconds']),
        index=holiday.loc[(~holiday['vacances'])&(holiday['Weekday']==smooth_from),'date'].tolist(),
        columns=holiday.loc[(holiday['vacances'])&(holiday['Weekday']==smooth_til),'date'].tolist()))

        if smooth_from=='Dimanche' and smooth_til=='Lundi':
            dict_smooth=dict(holiday_distsmooth.apply(lambda x: diff_smooth_1(x),axis=1))
            modify_smooth=[x for x in dict_smooth if dict_smooth[x]==1]
        elif smooth_from=='Samedi' and smooth_til=='Lundi':
            dict_smooth=dict(holiday_distsmooth.apply(lambda x: diff_smooth_1(x),axis=1))
            modify_smooth=[x for x in dict_smooth if dict_smooth[x]==2]  
        elif smooth_from=='Samedi' and smooth_til=='Vendredi':
            dict_smooth=dict(holiday_distsmooth.apply(lambda x: diff_smooth_2(x),axis=1))
            modify_smooth=[x for x in dict_smooth if dict_smooth[x]==1]  
        elif smooth_from=='Dimanche' and smooth_til=='Vendredi':
            dict_smooth=dict(holiday_distsmooth.apply(lambda x: diff_smooth_2(x),axis=1))
            modify_smooth=[x for x in dict_smooth if dict_smooth[x]==2]  

        holiday['vacances']=holiday.apply(lambda x: True if x[0] in modify_smooth else x[2],axis=1)

    def diff_left_right(x):
        dist_left=[y for y in x if y>0]
        dist_right=[y for y in x if y<0]
        if((len(dist_left)>0)&(len(dist_right)>0)):
            min_left=min(dist_left)
            min_right=max(dist_right)
            return [min_left,abs(min_right)]
        elif (len(dist_left)>0)&(len(dist_right)==0):
            min_left=min(dist_left)
            return [min_left,np.nan]
        elif (len(dist_left)==0)&(len(dist_right)>0):
            min_right=max(dist_right)
            return [np.nan,abs(min_right)]

    holiday_dist=np.round(DataFrame(np.subtract.outer(holiday.loc[holiday['vacances'],'seconds'],
    holiday.loc[~holiday['vacances'],'seconds']),
    index=holiday.loc[holiday['vacances'],'date'].tolist(),
    columns=holiday.loc[~holiday['vacances'],'date'].tolist()))

    holiday_dist=dict(holiday_dist.apply(lambda x: diff_left_right(x),axis=1))

    holiday['holiday_from_left']=holiday.apply(lambda x: holiday_dist[x[0]][0] if x[0] in holiday_dist else x[9],axis=1)
    holiday['holiday_from_right']=holiday.apply(lambda x: holiday_dist[x[0]][1] if x[0] in holiday_dist else x[10],axis=1)

    ###############################################

    holiday_dist=np.round(DataFrame(np.subtract.outer(holiday.loc[~holiday['vacances'],'seconds'],
    holiday.loc[holiday['vacances'],'seconds']),
    index=holiday.loc[~holiday['vacances'],'date'].tolist(),
    columns=holiday.loc[holiday['vacances'],'date'].tolist()))

    holiday_dist=dict(holiday_dist.apply(lambda x: diff_left_right(x),axis=1))

    holiday['holiday_to_left']=holiday.apply(lambda x: holiday_dist[x[0]][0] if x[0] in holiday_dist else x[11],axis=1)
    holiday['holiday_to_right']=holiday.apply(lambda x: holiday_dist[x[0]][1] if x[0] in holiday_dist else x[12],axis=1)

    ###############################################
    
    def diff_left_right(x):
        dist_left=[y for y in x if y>0]
        dist_right=[y for y in x if y<0]
        if((len(dist_left)>0)&(len(dist_right)>0)):
            min_left=min(dist_left)
            min_right=max(dist_right)
            return [[i for i in range(len(x)) if x[i]==min_left][0],
                [i for i in range(len(x)) if x[i]==min_right][0]]
        elif (len(dist_left)>0)&(len(dist_right)==0):
            min_left=min(dist_left)
            return [[i for i in range(len(x)) if x[i]==min_left][0],np.nan]
        elif (len(dist_left)==0)&(len(dist_right)>0):
            min_right=max(dist_right)
            return [np.nan,[i for i in range(len(x)) if x[i]==min_right][0]]

    holiday_dist=np.round(DataFrame(np.subtract.outer(holiday.loc[(~holiday['vacances']),'seconds'],
        holiday.loc[(holiday['vacances']),'seconds']),
        index=holiday.loc[(~holiday['vacances']),'date'].tolist(),
        columns=holiday.loc[(holiday['vacances']),'nom_vacances'].tolist()))
    holiday_dist2=holiday_dist.apply(lambda x: diff_left_right(x),axis=1)

    holiday_dist=dict(zip(holiday_dist2.keys(),[[holiday_dist.columns[x[0]],holiday_dist.columns[x[1]]] for x in holiday_dist2.values]))

    holiday['whichholiday_left']=holiday.apply(lambda x: holiday_dist[x[0]][0] if x[0] in holiday_dist else x[8],axis=1)
    holiday['whichholiday_right']=holiday.apply(lambda x: holiday_dist[x[0]][1] if x[0] in holiday_dist else x[13],axis=1)

    def diff_abs(x):
        min_abs=min(abs(x))
        return [[i for i in range(len(x)) if x[i]==min_abs][0],
            [i for i in range(len(x)) if x[i]==min_abs][0]]

    holiday_dist=np.round(DataFrame(np.subtract.outer(holiday.loc[(holiday['vacances']),'seconds'],
        holiday.loc[(holiday['vacances']),'seconds']),
        index=holiday.loc[(holiday['vacances']),'date'].tolist(),
        columns=holiday.loc[(holiday['vacances']),'nom_vacances'].tolist()))
    holiday_dist2=holiday_dist.apply(lambda x: diff_abs(x),axis=1)

    holiday_dist=dict(zip(holiday_dist2.keys(),[[holiday_dist.columns[x[0]],holiday_dist.columns[x[1]]] for x in holiday_dist2.values]))
    
    holiday['whichholiday_left']=holiday.apply(lambda x: holiday_dist[x[0]][0] if x[0] in holiday_dist else x[8],axis=1)
    holiday['whichholiday_right']=holiday.apply(lambda x: holiday_dist[x[0]][1] if x[0] in holiday_dist else x[13],axis=1)    

    ###############################################
    
    def diff_left_right(x):
        dist_left=[y for y in x if y>=0]
        dist_right=[y for y in x if y<=0]
        if((len(dist_left)>0)&(len(dist_right)>0)):
            min_left=min(dist_left)
            min_right=max(dist_right)
            return [min_left,abs(min_right)]
        elif (len(dist_left)>0)&(len(dist_right)==0):
            min_left=min(dist_left)
            return [min_left,np.nan]
        elif (len(dist_left)==0)&(len(dist_right)>0):
            min_right=max(dist_right)
            return [np.nan,abs(min_right)]

    holiday_dist=np.round(DataFrame(np.subtract.outer(holiday['seconds'],
    holiday.loc[holiday['est_jour_ferie'],'seconds']),
    index=holiday['date'].tolist(),
    columns=holiday.loc[holiday['est_jour_ferie'],'date'].tolist()))

    holiday_dist=dict(holiday_dist.apply(lambda x: diff_left_right(x),axis=1))

    holiday['jf_left']=holiday.apply(lambda x: holiday_dist[x[0]][0] if x[0] in holiday_dist else x[15],axis=1)
    holiday['jf_right']=holiday.apply(lambda x: holiday_dist[x[0]][1] if x[0] in holiday_dist else x[16],axis=1)    

    ###############################################

    def diff_left_right(x):
        dist_left=[y for y in x if y>=0]
        dist_right=[y for y in x if y<=0]
        if((len(dist_left)>0)&(len(dist_right)>0)):
            min_left=min(dist_left)
            min_right=max(dist_right)
            return [[i for i in range(len(x)) if x[i]==min_left][0],
                [i for i in range(len(x)) if x[i]==min_right][0]]
        elif (len(dist_left)>0)&(len(dist_right)==0):
            min_left=min(dist_left)
            return [[i for i in range(len(x)) if x[i]==min_left][0],0]
        elif (len(dist_left)==0)&(len(dist_right)>0):
            min_right=max(dist_right)
            return [0,[i for i in range(len(x)) if x[i]==min_right][0]]

    holiday_dist=np.round(DataFrame(np.subtract.outer(holiday['seconds'],
        holiday.loc[(holiday['est_jour_ferie']),'seconds']),
        index=holiday['date'].tolist(),
        columns=holiday.loc[(holiday['est_jour_ferie']),'nom_jour_ferie'].tolist()))
    holiday_dist2=holiday_dist.apply(lambda x: diff_left_right(x),axis=1)

    holiday_dist=dict(zip(holiday_dist2.keys(),[[holiday_dist.columns[x[0]],holiday_dist.columns[x[1]]] for x in holiday_dist2.values]))

    holiday['whichjf_left']=holiday.apply(lambda x: holiday_dist[x[0]][0] if x[0] in holiday_dist else x[14],axis=1)
    holiday['whichjf_right']=holiday.apply(lambda x: holiday_dist[x[0]][1] if x[0] in holiday_dist else x[17],axis=1)
    
    ###############################################
    
    holiday=holiday[['date']+[x for x in holiday.columns if 'holiday' in x or 'jf' in x]]

    holiday=holiday.rename(columns=dict(zip([x for x in holiday.columns if 'holiday' in x or 'jf' in x],
    [x+'_'+country for x in holiday.columns if 'holiday' in x or 'jf' in x])))

    Holiday=Holiday.merge(holiday,on='date')

In [628]:
Holiday

,date,weekday,yday,whichholiday_left_LUX,holiday_from_left_LUX,holiday_from_right_LUX,holiday_to_left_LUX,holiday_to_right_LUX,whichholiday_right_LUX,whichjf_left_LUX,jf_left_LUX,jf_right_LUX,whichjf_right_LUX
0,2017-01-01,Sunday,1,Vacances de Noeel,NaN,8.0,0.0,0.0,Vacances de Noeel,Nouvel an,0.0,0.0,Nouvel an
1,2017-01-02,Monday,2,Vacances de Noeel,NaN,7.0,0.0,0.0,Vacances de Noeel,Nouvel an,1.0,104.0,Peques
2,2017-01-03,Tuesday,3,Vacances de Noeel,NaN,6.0,0.0,0.0,Vacances de Noeel,Nouvel an,2.0,103.0,Peques
3,2017-01-04,Wednesday,4,Vacances de Noeel,NaN,5.0,0.0,0.0,Vacances de Noeel,Nouvel an,3.0,102.0,Peques
4,2017-01-05,Thursday,5,Vacances de Noeel,NaN,4.0,0.0,0.0,Vacances de Noeel,Nouvel an,4.0,101.0,Peques
5,2017-01-06,Friday,6,Vacances de Noeel,NaN,3.0,0.0,0.0,Vacances de Noeel,Nouvel an,5.0,100.0,Peques
6,2017-01-07,Saturday,7,Vacances de Noeel,NaN,2.0,0.0,0.0,Vacances de Noeel,Nouvel an,6.0,99.0,Peques
7,2017-01-08,Sunday,8,Vacances de Noeel,NaN,1.0,0.0,0.0,Vacances de Noeel,Nouvel an,7.0,98.0,Peques
8,2017-01-09,Monday,9,Vacances de Noeel,0.0,0.0,1.0,40.0,vacances de Carnaval,Nouvel an,8.0,97.0,Peques
9,2017-01-10,Tuesday,10,Vacances de Noeel,0.0,0.0,2.0,39.0,vacances de Carnaval,Nouvel an,9.0,96.0,Peques


In [629]:
Data=Data.merge(Holiday[[x for x in Holiday.columns if x!='weekday']],left_on='calendar_date',right_on='date')

In [632]:
Data=Data.loc[Data['calendar_date']<=pd.to_datetime('2019-07-15'),:]

In [634]:
Data.to_csv('R_Input.csv',index=False)